In [24]:

import requests
import chromadb
from base64 import b64encode
from common.payload import data2id
from common.article_scraper import ArticleScraper
from vector_db.utils import ImageEmbeddingFunction, TextEmbeddingFunction, format_document



client = chromadb.HttpClient(host="localhost",port=8000)

#Initialise Image Vector Database
text_fn = TextEmbeddingFunction(remote=True)
text_collection = client.get_or_create_collection(name="text_collection",embedding_function=text_fn)

#Initialise Image Vector Database
img_fn = ImageEmbeddingFunction()
img_collection = client.get_or_create_collection(name="img_collection",embedding_function=img_fn)

headers = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36"}

#To get query articles
articleScraper = ArticleScraper()

img_url = "https://cdn-attachments.timesofmalta.com/7b5555e18fb677fc34aa52cfa5c6c4332f500f2e-1715342220-dce7604f-960x640.jpg"
img_data = requests.get(img_url,
                        headers=headers).content
img_bytestring = b64encode(img_data).decode("ascii")


In [29]:
text_collection.get(ids="60a1b1cf2120a03c06f81cdeb15a3caf")

{'ids': ['60a1b1cf2120a03c06f81cdeb15a3caf'],
 'embeddings': None,
 'metadatas': [{'body': 'Former book council chair Mark Camilleri is in discussions with the government over his possible return to the council as a consultant, the author told Times of Malta\xa0on Friday.\n “There are problems in the council, and I will go and do the job that has been lacking for the past three years,” Camilleri said.\n “Nothing has yet been decided. I have not signed anything, and discussions are still ongoing.”\n His comments come after MaltaToday reported that Camilleri will return to the council as a consultant and will be paid a salary in the range of €30,000 a year. Sources told\xa0 MaltaToday he will be allowed to work remotely.\n Camilleri led the National Book Council for several years until his contract was not renewed by the Education Ministry in 2021, a year after he clashed with then-education minister Justyne Caruana.\n Since then, he has been highly critical of the council and his succes

In [25]:
a = img_collection.query(query_embeddings=img_fn(img_bytestring))

!
@@


In [28]:
a['metadatas']

[[{'article_ids': '["60a1b1cf2120a03c06f81cdeb15a3caf"]',
   'captions': '["Blogger and former National Book Council chairperson Mark Camilleri in 2023. Photo: Matthew Mirabelli"]',
   'selectors': '[".ar-ArticleHeader-Standard_sub > picture > img"]'},
  {'article_ids': '["09fa18abf1f361721d6795353f9f9479"]',
   'captions': '["The previously secret contents of Ryan Spagnol\'s lucrative cabinet contract have been revealed."]',
   'selectors': '["div.featured_posts img"]'},
  {'article_ids': '["554a51c3c34a7ae543bf1ba615fa7d3c"]',
   'captions': '["Joseph Muscat leaving court last week. Photo: Matthew Mirabelli"]',
   'selectors': '[".ar-ArticleHeader-Standard_sub > picture > img"]'},
  {'article_ids': '["82a5e56f7672e8f24d93aa2c3161df05"]',
   'captions': '["Kurt Buhagiar with his lawyers."]',
   'selectors': '["div.featured_posts img"]'},
  {'article_ids': '["4ec3df065cf70e541def77cd761fbcce"]',
   'captions': '["Prime Minister Robert Abela\\u2019s Chief of Staff Glenn Micallef and his

In [6]:
query_payload = articleScraper.scrape(
    "https://timesofmalta.com/article/man-dies-traffic-accident-swieqi.1090488"
)
if query_payload.error:
    print(query_payload.error)

query_document = format_document(query_payload.data,query=True)    

x = text_collection.query(
    query_texts=query_document
)
for d,m in zip(x['distances'][0],x['metadatas'][0]):
    print(f"{round(d,2)}\t{m['title']} - {m['url']}")

0.14	Man dies after traffic accident in Swieqi - https://timesofmalta.com/article/man-dies-traffic-accident-swieqi.1090488
0.22	Man dies after Swieqi car crash - https://www.maltatoday.com.mt//news/national/128475/man_dies_after_swieqi_car_crash
0.4	Man injured in Żebbuġ accident - https://timesofmalta.com/article/man-injured-zebbug-accident.1087372
0.42	Man in hospital after traffic accident in Regional Road - https://www.maltatoday.com.mt//news/national/127462/man_in_hospital_after_traffic_accident_in_regional_road
0.44	Man, 35, hit by car in Swieqi - https://timesofmalta.com/article/man-35-hit-car-swieqi.1041104
0.46	Two men injured in separate traffic accidents - https://timesofmalta.com/article/two-men-injured-separate-traffic-accidents.1031498
0.47	Driver seriously injured in three-vehicle crash in St Julian's - https://timesofmalta.com/article/driver-seriously-injured-threevehicle-crash-st-julian.1083528
0.49	Motorcyclist, 73, dies at hospital following Żebbuġ crash - https://ti